In [ ]:
import ee
ee.Authenticate()

In [455]:
ee.Initialize()

In [460]:
hc_img = ee.Image('projects/wri-datalab/HansenComposite_14-15')

In [473]:
# Import Google Earth API
import ee

# Import the Folium library.
import folium

# Define a method for displaying Earth Engine image tiles to folium map.
def add_ee_layer(self,ee_image_object,vis_params, name):
  map_id_dict = ee.Image(ee_image_object).getMapId(vis_params)
  folium.raster_layers.TileLayer(
    tiles = map_id_dict['tile_fetcher'].url_format,
    attr = 'Map Data &copy; <a href="https://earthengine.google.com/">Google Earth Engine</a>',
    name = name,
    overlay = True,
    control = True
  ).add_to(self)

# Add EE drawing method to folium.
folium.Map.add_ee_layer = add_ee_layer

# Create a folium map object.
my_map = folium.Map(location=[20, 0], zoom_start=3, height=500)

# Add the elevation model to the map object.
gfc2015 = ee.Image('UMD/hansen/global_forest_change_2015')

# Cover Layer
cover = gfc2015.select('treecover2000')
my_map.add_ee_layer(cover.mask(cover),{
  'bands': ['treecover2000'],
  'max': [100],
  'palette': ['000000', '00FF00'],
},'treecover2000')

# Loss Layer
loss = gfc2015.select('loss')
my_map.add_ee_layer(loss.mask(loss),{
  'bands': ['loss'],
  'min' : [1],
  'max' : [15],
  'palette' : ['red']
},'loss')

# Add a layer control panel to the map.
my_map.add_child(folium.LayerControl())

# Display the map.
display(my_map)

In [ ]:

boundary = ee.Geometry.Rectangle([116.2621, 39.8412, 116.4849, 40.01236])

task_config = {
    'fileNamePrefix': 'image',
    'crs': 'EPSG:4326',
    'scale': 500,
    # 'maxPixels': 100000000000000,
    'fileFormat': 'GeoTIFF',
    'skipEmptyTiles': True,
    'region': boundary ,
    'folder': 'myfolder'
    }

image = ee.Image('UMD/hansen/global_forest_change_2015')
task = ee.batch.Export.image.toDrive(image, str('image'), **task_config)
task.start()

In [462]:
# import ee
# ee.Initialize()
# hc_img=ee.Image('projects/wri-datalab/HansenComposite_14-15')
# tc_ic=ee.ImageCollection('projects/wri-datalab/HansenByTreecover')
# annual_ic=ee.ImageCollection('projects/wri-datalab/AnnualHansen')
# geoms=ee.FeatureCollection('ft:13BvM9v1Rzr90Ykf1bzPgbYvbb8kGSvwyqyDwO8NI')


# from time import time

# def profile(func=None,N=1,**kwargs):
#     start=time()
#     print("\n\tfunction:",func.__name__)
#     prin("\targs:",kwargs)
#     for i in range(N): print("\tresult ({}/{}):".format(i+1,N),func(**kwargs))
#     print ("\ttime:",(time()-start),"\n")



# FULL_SCALE=27.829872698318393
# REDUCED_SCALE=90
# CRS="EPSG:4326"



# def getgeom(name):
#     return ee.Feature(geoms.filter(ee.Filter.eq('name',name)).first()).geometry()


# def count_reducer(img,geom,scale):
#     return img.mask(img).reduceRegion(
#             reducer=ee.Reducer.count().unweighted(),
#             scale=scale,
#             crs=CRS,
#             geometry=getgeom(geom),
#             maxPixels=25000000000
#         )


# def sum_reducer(img,geom,scale):
#     return img.reduceRegion(
#             reducer=ee.Reducer.sum().unweighted(),
#             scale=scale,
#             crs=CRS,
#             geometry=getgeom(geom),
#             maxPixels=25000000000
#         )


# #
# # MultiYear
# #
# def hc(threshold=None,ymin=None,ymax=None,geom=None,scale=None,reducer_func=None):
#     loss=hc_img.select(['loss_{}'.format(threshold)])
#     maskimg=loss.gte(ymin).And(loss.lte(ymax))
#     return reducer_func(maskimg,geom,scale).getInfo()


# def treecover1(threshold=None,ymin=None,ymax=None,geom=None,scale=None,reducer_func=None):
#     loss=ee.Image('projects/wri-datalab/HansenByTreecover/threshold_loss-{}'.format(threshold))
#     maskimg=loss.select(map(lambda y: 'loss_{}'.format(y),range(ymin,ymax+1))).reduce(ee.Reducer.max())
#     return reducer_func(maskimg,geom,scale).getInfo()


# def treecover2(threshold=None,ymin=None,ymax=None,geom=None,scale=None,reducer_func=None):
#     loss=ee.Image(tc_ic.filter(ee.Filter.eq('treecover2000',threshold)).max())
#     maskimg=loss.select(map(lambda y: 'loss_{}'.format(y),range(ymin,ymax+1))).reduce(ee.Reducer.max())
#     return reducer_func(maskimg,geom,scale).getInfo()


# def annual(threshold=None,ymin=None,ymax=None,geom=None,scale=None,reducer_func=None):
#     loss=annual_ic.select(['loss_{}'.format(threshold)])
#     maskimg=ee.Image(loss.filter(ee.Filter.inList('year',ee.List.sequence(ymin,ymax))).reduce(ee.Reducer.max()))
#     return reducer_func(maskimg,geom,scale).getInfo()